In [15]:
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 

In [16]:
cwd = os.chdir(r'C:\Users\rsurs\OneDrive\Documents\Zerodha\api_keys')
#print(f"current dir: {os.getcwd()}")
request_token = open('request_token.txt','r').read()
key_secret = open('credentials.txt','r').read().split()
access_token = open('access_token.txt','r').read()
api_key = key_secret[0]
api_secret = key_secret[1]
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [28]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [32]:
instrument_df.head()


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [19]:
def instrument_token(instrument_df, symbol):
    """
    This function will return the token number of the instrument from data
    """
    return instrument_df[instrument_df.tradingsymbol == symbol].instrument_token.values[0]

In [26]:
instrument_token(instrument_df,'RELIANCE')

738561

In [23]:
def fetchOHLC(ticker,interval,duration):
    instrument = instrument_token(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [33]:
ohlc = fetchOHLC('ICICIBANK','5minute',5)
ohlc.head()

,open,high,low,close,volume
date,,,,,
2024-05-23 09:15:00+05:30,1115.55,1117.70,1113.05,1116.35,410513
2024-05-23 09:20:00+05:30,1116.30,1117.50,1114.75,1116.25,172455
2024-05-23 09:25:00+05:30,1116.20,1116.90,1112.00,1112.45,220070
2024-05-23 09:30:00+05:30,1112.45,1114.25,1111.40,1114.00,166897
2024-05-23 09:35:00+05:30,1114.05,1115.70,1113.80,1115.00,272245


In [37]:
# def MACD(ohlc,a,b,c): # a is fast ma (12 days), b is slow ma(26 day), c (9 days) is the ma difference between fast ma(a) and slow ma(b)                        #
#     df = ohlc.copy() # Note to zerodha uses exponential moving average and in our code we used simple moving average
#     # to calculate moving average use rolling().mean()
#     df['MA_Fast'] = df['close'].rolling(a).mean()
#     df['MA_Slow'] = df['close'].rolling(b).mean()
#     df['MACD'] = df['MA_Fast'] - df['MA_Slow']
#     df['Signal'] = df['MACD'].rolling(c).mean()
#     df.dropna(inplace=True)
#     return df

In [39]:
def MACD(ohlc,a,b,c): # We are using exponential moving avg
    df = ohlc.copy()
    df['MA_Fast'] = df['close'].ewm(span=a,min_periods=a).mean()
    df['MA_Slow'] = df['close'].ewm(span=b,min_periods=b).mean()
    df['MACD'] = df['MA_Fast'] - df['MA_Slow']
    df['Signal'] = df['MACD'].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return df

In [40]:
macd_df =  MACD(ohlc,a=12,b=26,c=9)
macd_df.head()

,open,high,low,close,volume,MA_Fast,MA_Slow,MACD,Signal
date,,,,,,,,,
2024-05-23 12:00:00+05:30,1126.30,1127.45,1124.50,1124.85,670741,1125.507545,1125.002594,0.504950,0.641373
2024-05-23 12:05:00+05:30,1124.85,1125.50,1124.45,1125.00,63242,1125.429235,1125.002388,0.426846,0.593307
2024-05-23 12:10:00+05:30,1125.00,1125.50,1124.15,1124.80,68814,1125.332192,1124.986395,0.345797,0.539153
2024-05-23 12:15:00+05:30,1124.85,1125.90,1124.65,1124.90,115715,1125.265563,1124.979601,0.285962,0.484778
2024-05-23 12:20:00+05:30,1125.00,1125.45,1124.55,1124.65,70256,1125.170695,1124.953801,0.216894,0.428085
